In [ ]:
import pandas as pd
import os
import numpy as np

"""
📊 **SNR Results Export to Excel**
This script computes and saves SNR results into an organized Excel file.
"""

def save_snr_to_excel(images, probe, repo_directory):
    """
    Computes and saves SNR values into an Excel file, organized by probe type.

    📌 **How it works**:
    - Extracts **Signal-to-Noise Ratio (SNR)** from each image.
    - Saves results in an Excel file inside a **dedicated folder (`snr_results`)**.
    - Creates separate sheets for each metabolite.

    🔹 **Parameters**:
    - `images` (dict): Dictionary with image filenames as keys and image data as values.
    - `probe` (str): The selected probe (`p1` for 3FDG, `p2` for 3FDGal).
    - `repo_directory` (str): Path to the main project directory.

    📂 **Output**:
    - `snr_results/SNR_Results_P1.xlsx` (for 3FDG) or
    - `snr_results/SNR_Results_P2.xlsx` (for 3FDGal)
    """

    # 📂 **Ensure results folder exists**
    snr_results_dir = os.path.join(repo_directory, "snr_results")
    os.makedirs(snr_results_dir, exist_ok=True)

    # 🎯 **Probe-Specific Mappings**
    metabolite_map = {
        "p1": {"m1": "3FDGlucose", "m3": "3FDGluconic Acid", "m4": "3FDSorbitol"},
        "p2": {"m1": "3FDGalactose", "m3": "3FDGalactonic Acid", "m4": "3FDGalactitol"}
    }
    time_map = {"t1": "1hr", "t2": "2hr", "t3": "3hr"}

    # 🗂 **Prepare Data Storage**
    metabolite_data = {met: [] for met in metabolite_map[probe].values()}
    summary_data = []

    for file_path, img in images.items():
        snr, max_signal, noise, _ = compute_snr(np.array(img))

        # 🏷 Extract filename details
        parts = file_path.replace(".tif", "").split("_")
        if len(parts) < 4:
            continue  # Skip unexpected formats

        time_label = time_map.get(parts[2], parts[2])
        metabolite_label = metabolite_map[probe].get(parts[3], parts[3])

        # 📌 **Store Data**
        metabolite_data[metabolite_label].append([file_path, time_label, snr, max_signal, noise])
        summary_data.append([file_path, metabolite_label, time_label, snr, max_signal, noise])

    # 📄 **Create and Save Excel File**
    output_excel = os.path.join(snr_results_dir, f"SNR_Results_{probe.upper()}.xlsx")
    with pd.ExcelWriter(output_excel, engine="xlsxwriter") as writer:
        # 🔹 **Summary Sheet**
        summary_df = pd.DataFrame(summary_data, columns=["Filename", "Metabolite", "Time", "SNR", "Max Signal", "Noise Std Dev"])
        summary_df.to_excel(writer, sheet_name="Summary", index=False)

        # 🔹 **Separate Sheets for Each Metabolite**
        for met, data in metabolite_data.items():
            df = pd.DataFrame(data, columns=["Filename", "Time", "SNR", "Max Signal", "Noise Std Dev"])
            df.to_excel(writer, sheet_name=met, index=False)

    print(f"✅ **SNR results saved:** {output_excel}")

# 📝 **User Input: Define Paths**
repo_directory = r"YOUR_LOCAL_PATH_HERE/FMRI-Metabolism-Quantification-main"

# 🎯 **Set Probe Type ('p1' for 3FDG or 'p2' for 3FDGal)**
probe = "p2"

# 🔍 **Compute & Save SNR Data**
save_snr_to_excel(images, probe, repo_directory)
